In [1]:
import matplotlib as mpl
import data as Data
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
import pandas as pd
import os
import numpy as np
from cycler import cycler

# %matplotlib inline # for inline plots
%matplotlib qt

# Set Data Directories

In [2]:
root_dir = '/home/sam/nupyprop_test' # main directory where script files are located
data_dir = '/home/sam/nupyprop_test/output' # data directory where output hdf5 files are located
psurv_dir = '/home/sam/nupyprop_test/p_surv' # data directory where p_survival output files are located
plt.style.use(root_dir+'/matplotlibrc') # my custom plotting style

# Set desired parameters 

In [10]:
# energies = np.array([1e7, 1e8, 1e9, 1e10, 1e11]) # ingoing neutrino energy in GeV
energies = np.logspace(7,11,17)

nu_type = 'nu'
lepton = 'tau' # type of lepton

nu_tye = 'nu' # nu for neutrino & anu for anti-neutrino
nu_cs = 'ct18nlo' # neutrino cross-section model
lep_pn = 'allm' # photonuclear energy loss model
idepth = 4 # depth of water layer in km
stats = 1e8 # no. of ingoing neutrinos ie., statistics
loss_type = 'stochastic' # type of energy loss; can be stochastic or continuous

material = 'rock' # type of material used in propagation
beta_type = 'total' # type of beta required; can be cut or total

## Function to get p_exit values

In [4]:
def p_exit(nu_type, lepton, energies, reg, idepth, nu_cs, lep_pn, type_loss, stats):
    os.chdir(data_dir)
    p_exit_dict = {}
    for energy in energies:
        p_exit = Data.get_pexit(nu_type, lepton, energy, reg, idepth, nu_cs, lep_pn, type_loss, stats)
        lists = sorted(p_exit.items())
        x,y = zip(*lists)
        p_exit_dict[energy] = np.asarray(y)
    return p_exit_dict

## Function to plot exit probability Vs. earth emergence angles

In [14]:
def plot_npp(nu_type, lepton, energies, idepth, nu_cs, lep_pn, loss_type, stats):
    p_exit_dict = p_exit(nu_type, lepton, energies, 'regen', idepth, nu_cs, lep_pn, loss_type, stats)
    
    angles = np.arange(1,36)
    
    if len(energies) > 4:
        cc = (cycler(color=list('rgbkm')) * cycler(linestyle=['solid', 'dotted', 'dashed', 'dashdot']))
    else:
        cc = (cycler(color=list('rgbkm')))
              
    params = {'axes.prop_cycle': cc}
    plt.rcParams.update(params)

    fig, axs = plt.subplots(figsize=(20,10),dpi=100)
#     color_lst = ['r', 'g', 'b', 'k', 'm']

    for i in range(len(energies)):
        energy_log = int(np.log10(energies[i]))

        axs.loglog(angles, p_exit_dict[energies[i]], linewidth = 2, label = '%.0d' % energy_log)


        axs.legend(loc='best',ncol=int(len(energies)/2), title = r'log$_{10}(E_{\nu}/GeV)$', frameon=False, framealpha=0.5)


    # ticks = (1,2,3,4,5,6,7,8,9,10,20,30,40,50)
    # ticks = (1,2,3,4,5,6,7,8,9,10,15,20,25,30,35)
    ticks = (1,2,3,4,5,10,15,20,25,30,35)
    axs.set_xticks(ticks)
    axs.set_xticklabels([i for i in ticks])


    axs.grid(which='major', axis='both', linestyle='--')
    axs.grid(which='minor', linestyle=':', linewidth='0.2', color='black')
    axs.xaxis.set_ticks_position('both')
    axs.yaxis.set_ticks_position('both')
    axs.tick_params(axis='x', which='both', labelbottom = True, labeltop = False)
    axs.tick_params(axis='y', which='both', left = True, labelleft = True, labelright= True)
    for axis in [axs.xaxis]:
        axis.set_major_formatter(ScalarFormatter())
    axs.set_xlabel("Emergence Angle (Deg)")

    axs.set_ylabel(r"$P_{exit}$")
    axs.set_ylim(1e-6,1e-1)

    axs.set_xlim(1,35)

    plt.tight_layout()
    # plt.savefig('p_exit_npp.png', format='png', dpi = 100)

    return None

In [15]:
plot_npp(nu_type, lepton, energies, idepth, nu_cs, lep_pn, loss_type, stats)